# Підготовка та Аналіз даних
## Лабораторна робота №2
## Наука про дані: підготовчий етап
<b>Мета роботи</b>: ознайомитися з основними кроками по роботі з даними –
workflow від постановки задачі до написання пояснювальної записки,
зрозуміти постановку задачі та природу даних, над якими виконується
аналітичні операції
#### ФБ-23 Хоменко Гліб

In [1]:
import urllib.request
import pandas as pd
import os
from datetime import datetime

Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс(номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження;

In [2]:
def download(index):
    url=f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={index}&year1=1981&year2=2024&type=Mean"
    now = datetime.now()
    date_and_time = now.strftime("%d-%m-%Y_%H-%M-%S")
    vhi_url = urllib.request.urlopen(url)
    out = open('NOAA/NOAA_ID_'+str(index)+'_'+date_and_time+'.csv','wb')
    out.write(vhi_url.read())
    out.close()
    print("VHI is downloaded...")
for i in range(1, 28):
    download(i)


VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...
VHI is downloaded...


Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах).
Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо).Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [3]:
def frame(path):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    all_dataframes = []
    
    for filename in os.listdir(path):
        if not filename.endswith('.csv'):
            continue
        tokens = filename.split("_")
        if tokens[0] != "NOAA":
            continue
        index = tokens[2]
        filepath = os.path.join(path, filename)
            
        df = pd.read_csv(filepath, header=1, names=headers)
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df['Region'] = index
        all_dataframes.append(df)
            
    combined_df = pd.concat(all_dataframes, ignore_index=True).drop_duplicates().reset_index(drop=True)
    combined_df['Year'] = combined_df['Year'].str.replace('<tt><pre>', '').str.replace('</pre></tt>', '')
    combined_df = combined_df[combined_df['Year'].astype(str).str.strip() != '']
    combined_df['Year'] = combined_df['Year'].astype(int)
    combined_df['Region'] = combined_df['Region'].astype(int)
    combined_df = combined_df.drop(columns=['empty'])
    return combined_df

combined_dataframe = frame('NOAA')
combined_dataframe


,Year,Week,SMN,SMT,VCI,TCI,VHI,Region
0,1982,1.0,0.059,258.24,51.11,48.78,49.95,10
1,1982,2.0,0.063,261.53,55.89,38.20,47.04,10
2,1982,3.0,0.063,263.45,57.30,32.69,44.99,10
3,1982,4.0,0.061,265.10,53.96,28.62,41.29,10
4,1982,5.0,0.058,266.42,46.87,28.57,37.72,10
...,...,...,...,...,...,...,...,...
58234,2024,6.0,0.112,277.44,62.16,5.41,33.78,9
58235,2024,7.0,0.123,279.48,63.24,4.80,34.02,9
58236,2024,8.0,0.136,281.56,62.98,4.84,33.91,9
58237,2024,9.0,0.148,283.39,62.66,6.01,34.33,9


Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA

In [14]:
combined_dataframe["Region"].replace({24:1, 25:2, 5:3, 6:4, 27:5, 23:6, 26:7, 7:8, 11:9, 13:10, 14:11, 15:12, 16:13, 17:14,
18:15, 19:16, 21:17, 22:18, 8:19, 9:20, 10:21, 1:22, 3:23, 2:24, 4:25, 12:26, 20:27}, inplace = True)

combined_dataframe

,Year,Week,SMN,SMT,VCI,TCI,VHI,Region
0,1982,1.0,0.059,258.24,51.11,48.78,49.95,9
1,1982,2.0,0.063,261.53,55.89,38.20,47.04,9
2,1982,3.0,0.063,263.45,57.30,32.69,44.99,9
3,1982,4.0,0.061,265.10,53.96,28.62,41.29,9
4,1982,5.0,0.058,266.42,46.87,28.57,37.72,9
...,...,...,...,...,...,...,...,...
58234,2024,6.0,0.112,277.44,62.16,5.41,33.78,23
58235,2024,7.0,0.123,279.48,63.24,4.80,34.02,23
58236,2024,8.0,0.136,281.56,62.98,4.84,33.91,23
58237,2024,9.0,0.148,283.39,62.66,6.01,34.33,23


Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [15]:
def minmax(df, index, year):
    vhi = df[(df["Region"] == index) & (df["Year"] == year)]
    min_v = vhi['VHI'].min()
    max_v = vhi['VHI'].max()
#     print(vhi)
    print("min VHI:", min_v)
    print("max VHI:", max_v)
for i in range(1, 28):
    minmax(combined_dataframe, i, 2000)


min VHI: 17.4
max VHI: 69.82
min VHI: 18.83
max VHI: 60.15
min VHI: nan
max VHI: nan
min VHI: 27.46
max VHI: 66.3
min VHI: nan
max VHI: nan
min VHI: nan
max VHI: nan
min VHI: nan
max VHI: nan
min VHI: 22.52
max VHI: 68.24
min VHI: 20.97
max VHI: 65.0
min VHI: 18.98
max VHI: 60.12
min VHI: 16.84
max VHI: 60.69
min VHI: 11.25
max VHI: 63.27
min VHI: 26.15
max VHI: 59.59
min VHI: 16.14
max VHI: 60.4
min VHI: 15.17
max VHI: 67.16
min VHI: nan
max VHI: nan
min VHI: 23.86
max VHI: 67.04
min VHI: 24.65
max VHI: 68.85
min VHI: 29.53
max VHI: 69.96
min VHI: 16.81
max VHI: 66.21
min VHI: 9.36
max VHI: 68.45
min VHI: 23.43
max VHI: 74.45
min VHI: 8.14
max VHI: 71.94
min VHI: 20.12
max VHI: 55.16
min VHI: 17.77
max VHI: 61.55
min VHI: 10.68
max VHI: 72.05
min VHI: 10.6
max VHI: 74.06


Ряд VHI за вказаний діапазон років для вказаних областей; виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25);

In [6]:
def drought(df, regions, start_year, end_year, percent):
    for region in regions:
        print(f"Region {region}:")
        vhi = df[(df['Region'].isin(regions)) & (df['Year'].isin(list(range(start_year, end_year + 1))))]
        if not vhi.empty:
            print(vhi)
        else:
            print(f"No data available for region {region} for year {year}")
    print("\n")
        
    total_regions = df['Region'].nunique()
    drought_years = []
    for year in range(df['Year'].min(), df['Year'].max() + 1):
        affected_regions = df[(df['Year'] == year) & (df['VHI'] < 15) & (df['VHI'] != -1)]['Region'].nunique()
        if affected_regions > total_regions * (percent / 100):
            drought_years.append(year)
    
    print(f"Years with extreme drought affecting more than {percent}% of the regions in Ukraine:")
    print(drought_years)
drought(combined_dataframe, [20, 21], 2020, 2024, 10)


Region 20:
       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
1926   2020   1.0  0.082  268.63  72.97   8.14  40.56      21
1927   2020   2.0  0.083  268.63  75.61   9.99  42.80      21
1928   2020   3.0  0.083  268.26  76.47  12.72  44.59      21
1929   2020   4.0  0.086  268.85  77.06  13.32  45.19      21
1930   2020   5.0  0.090  270.17  73.45  14.60  44.02      21
...     ...   ...    ...     ...    ...    ...    ...     ...
58234  2024   6.0  0.112  277.44  62.16   5.41  33.78      20
58235  2024   7.0  0.123  279.48  63.24   4.80  34.02      20
58236  2024   8.0  0.136  281.56  62.98   4.84  33.91      20
58237  2024   9.0  0.148  283.39  62.66   6.01  34.33      20
58238  2024  10.0  0.157  284.83  59.79   9.18  34.48      20

[690 rows x 8 columns]
Region 21:
       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
1926   2020   1.0  0.082  268.63  72.97   8.14  40.56      21
1927   2020   2.0  0.083  268.63  75.61   9.99  42.80      21
1928   2020   3.0  0.083

Аналогічно для помірних посух

In [7]:
def moderate_drought(df, regions, start_year, end_year, percent):
    for region in regions:
        print(f"Region {region}:")
        vhi = df[(df['Region'].isin(regions)) & (df['Year'].isin(list(range(start_year, end_year + 1))))]
        if not vhi.empty:
            print(vhi)
        else:
            print(f"No data available for region {region} for year {year}")
    print("\n")
        
    total_regions = df['Region'].nunique()
    drought_years = []
    for year in range(df['Year'].min(), df['Year'].max() + 1):
        affected_regions = df[(df['Year'] == year) & (df['VHI'] < 15) & (df['VHI'] != -1)]['Region'].nunique()
        if affected_regions > total_regions * (percent / 100):
            drought_years.append(year)
    
    print(f"Years with extreme drought affecting more than {percent}% of the regions in Ukraine:")
    print(drought_years)
moderate_drought(combined_dataframe, [20, 21], 2020, 2024, 10)

Region 20:
       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
1926   2020   1.0  0.082  268.63  72.97   8.14  40.56      21
1927   2020   2.0  0.083  268.63  75.61   9.99  42.80      21
1928   2020   3.0  0.083  268.26  76.47  12.72  44.59      21
1929   2020   4.0  0.086  268.85  77.06  13.32  45.19      21
1930   2020   5.0  0.090  270.17  73.45  14.60  44.02      21
...     ...   ...    ...     ...    ...    ...    ...     ...
58234  2024   6.0  0.112  277.44  62.16   5.41  33.78      20
58235  2024   7.0  0.123  279.48  63.24   4.80  34.02      20
58236  2024   8.0  0.136  281.56  62.98   4.84  33.91      20
58237  2024   9.0  0.148  283.39  62.66   6.01  34.33      20
58238  2024  10.0  0.157  284.83  59.79   9.18  34.48      20

[690 rows x 8 columns]
Region 21:
       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
1926   2020   1.0  0.082  268.63  72.97   8.14  40.56      21
1927   2020   2.0  0.083  268.63  75.61   9.99  42.80      21
1928   2020   3.0  0.083